# Official Vulnerability Viewer

In [1]:
import codecs
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import random
import colorama
from colorama import Fore, Back, Style
from IPython.display import clear_output
from ast import literal_eval

import settings
from utils import vplot, vprint

In [3]:
FRAMEWORKS = ['tensorflow','opencv','pytorch','keras', 'caffe']
DISTILLED_DIR = settings.DATA_CONFIG['distilled_dir']

MANUAL_DIR = settings.DATA_CONFIG['manual_dir']

#### Read Excel

In [4]:
def read_official_excel():
    filename = MANUAL_DIR + f'official_vulnerability.xlsx'
    return pd.read_excel(filename,sheet_name='manual')
def read_tf_security_advisory():
    filename = DISTILLED_DIR + f'security_advisory_tensorflow.csv'
    return pd.read_csv(filename)
    
df = read_official_excel()
tfsa_df = read_tf_security_advisory()
tfsa_df['CVE Number'] = tfsa_df['CVE Number'].str.strip()

#### Printer

In [5]:
def print_vuln(df, tfsa_df, cve):
    row = df.loc[df['CVE ID'] == cve].iloc[0]
   
    framework = row['framework'] # framework
    cwe = row['CWE ID']          # cwe
    # commit
    commits = row['commit hash']
    if commits is not np.nan:
        commits = commits.split(',')
    # cwe description
    cwe_desc = row['Description']
    # commit description
    commit_desc = row['commit description']
        
    print(f"CVE ID: {cve} | Framework: {framework} | CWE ID: {cwe}")
    print(f"\n## CWE Description\n{cwe_desc}\n")
    
    if framework == 'tensorflow':
        tfsa_row = tfsa_df.loc[tfsa_df['CVE Number'] == cve]
        if len(tfsa_row) > 0:
            tfsa_row = tfsa_row.iloc[0]
            tfsa = tfsa_row['TFSA']
            patches = tfsa_row['Patches']
            issues = tfsa_row['Issue Description']
            impact = tfsa_row['Impact']
            print(f"\n## TF Security Adversory\n{tfsa}\n")
            print(f"\n## Patches\n{patches}\n")
            print(f"\n## Issues\n{issues}\n")
            print(f"\n## Impact\n{impact}\n")
    else:
        if commits is not np.nan:
            for commit in commits:
                print(commit)
    print('-'*100)

In [6]:
opencv_df = df[df['framework']=='opencv']
opencv_df = opencv_df[~opencv_df['commit hash'].isnull()]

In [7]:
opencv_df

,CVE ID,framework,Description,commit hash,commit description,CWE ID,CVSS2 Access Complexity,CVSS2 Authentication Required,CVSS2 Availability Impact,CVSS2 Confidentiality Impact,...,CVSS3attackVector,CVSS3attackComplexity,CVSS3privilegesRequired,CVSS3userInteraction,CVSS3scope,CVSS3confidentialityImpact,CVSS3integrityImpact,CVSS3availabilityImpact,CVSS3baseScore,CVSS3baseSeverity
422,CVE-2017-17760,opencv,OpenCV 3.3.1 has a Buffer Overflow in the cv::...,7bbe1a53cfc097b82b1589f7915a2120de39274c,imgcodecs(pxm): fix memcpy size,CWE-119,MEDIUM,NONE,PARTIAL,NONE,...,NETWORK,LOW,NONE,REQUIRED,UNCHANGED,NONE,NONE,HIGH,6.5,MEDIUM
441,CVE-2019-10061,opencv,utils/find-opencv.js in node-opencv (aka OpenC...,"aaece6921d7368577511f06c94c99dd4e9653563, 81a4...",NaN,CWE-78,LOW,NONE,PARTIAL,PARTIAL,...,NETWORK,LOW,NONE,NONE,UNCHANGED,HIGH,HIGH,HIGH,9.8,CRITICAL
446,CVE-2019-19624,opencv,An out-of-bounds read was discovered in OpenCV...,d1615ba11a93062b1429fce9f0f638d1572d3418,video:fixed DISOpticalFlow segfault from small...,CWE-125,LOW,NONE,PARTIAL,PARTIAL,...,NETWORK,LOW,NONE,NONE,UNCHANGED,LOW,NONE,LOW,6.5,MEDIUM
